## 1.扩展断点处理

我们在第3章中用事件处理函数（如$\text{exception_handler_breakpoint}$和$\text{exception_handler_single_step}$）处理调试事件的方法，用PyDbg可以很容易地扩展这种功能，只需要构建一个用户模式的回调函数。当收到一个调试事件的时候，回调函数执行所定义的操作，如读取特定地址的数据，设置更多的断点，操作内存等等。待操作完成后，将权限交还给调试器，恢复被调试的进程。

PyDbg设置断点的函数原型如下：

$\text{bp_set(address, description="", restore=True, handler=None)}$

$\quad\bullet\quad$$\text{address}$是设置断点的内存地址

$\quad\bullet\quad$$\text{description}$参数可选，用于给每个断点设置唯一的名字

$\quad\bullet\quad$$\text{restore}$决定是否在断点被触发后重新设置

$\quad\bullet\quad$$\text{handler}$指向断点触发时调用的回调函数

断点回调函数只接受一个参数，即pydbg类的实例化对象；  
所有的上下文数据，线程和进程信息都在回调函数被调用的时候填装在这个类中。

以$\text{printf_loop.py}$为测试目标，让我们实现一个自定义的回调函数，这次我们在$\text{printf()}$函数上设置断点，以便读取$\text{printf()}$输出时用到的参数$\text{counter}$变量值，之后用一个1到100之间的随机数替换$\text{counter}$的值，并打印出来。

$\text{printf_loop.py}$

In [ ]:
from ctypes import *
import time

msvcrt  = cdll.msvcrt
counter = 0

while True:
    msvcrt.printf("Loop iteration %d!\n" % counter)
    time.sleep(2)
    counter += 1

新建$\text{printf_random.py}$

In [ ]:
from pydbg import *
from pydbg.defines import *

import struct
import random

# This is our user defined callback function
def printf_randomizer(dbg):

    # Read in the value of the counter at ESP + 0x8 as a DWORD
    parameter_addr = dbg.context.Esp + 0x8
    counter        = dbg.read_process_memory(parameter_addr, 4)

    # When we use read_process_memory, it returns a picked binary string.
    # We must first unpack it before we cat use it further.
    counter = struct.unpack("L", counter)[0]
    print "Counter: %d" % int(counter)

    # Generate a random number and pack it into binary format
    # so that it is written correctly back into the process.
    random_counter = random.randint(1, 100)
    random_counter = struct.pack("L", random_counter)[0]

    # Now swap in our random number and resume the process
    dbg.write_process_memory(parameter_addr, random_counter)

    return DBG_CONTINUE


# Instantiate the pydbg class
dbg = pydbg()

# Now enter the PID of the printf_loop.py process
pid = raw_input("Enter the printf_loop.py PID: ")

# Attach the debugger to that process
dbg.attach(int(pid))

# Set the breakpoint with the printf_randomizer function
# defined as a callback
printf_address = dbg.func_resolve("msvcrt", "printf")
dbg.bp_set(printf_address, description="printf_address",handler=printf_randomizer)

# Resume the process
dbg.run()

#### 说明：
在这里捣鼓了很久也没法运行，稍后再进行处理！

## 2.处理访问违例

当程序尝试访问它们没有权限访问的内存页面或者以一种不合法的方式访问内存的时候就会产生内存违例。  
导致内存违例错误的范围很广，从内存溢出到不恰当的处理空指针都有可能出现。  
从安全角度考虑，每一个访问违例都应该仔细审查，因为他们有可能会被利用。

当调试器处理访问违例的时候，需要搜集所有和违例相关的信息：栈框架、寄存器以及引起违例的指令。  
接着我们就能够用这些信息写一个利用程序或者创建一个二进制的补丁文件。

PyDbg能够很方便的实现一个违例访问处理函数，并输出相关的奔溃信息。  
我们下面来测试的目标是危险的$\text{C}$函数$\text{strcpy()}$，我们用$\text{strcpy()}$函数来创建一个会被溢出的程序。  
接下来我们再写一个简短的PyDbg脚本附加到进程并处理违例。

产生溢出的脚本--$\text{buffer_overflow.py}$

In [ ]:
from ctypes import *

msvcrt = cdll.msvcrt

# Give the debugger time to attach,
# then hit a button
raw_input("Once the debugger is attached, press any key.")

# Create the 5-byte destination buffer
buffer = c_char_p("AAAAA")

# The overflow string
overflow = "A" * 100

# Run the overflow
msvcrt.strcpy(buffer, overflow)

处理溢出的脚本--$\text{access_violation_handler.py}$

In [ ]:
from pydbg import *
from pydbg.defines import *

# Utility libraries included with PyDbg
import utils

# This is our access violation handler
def check_accessv(dbg):

    # We ship first-chance exceptions
    if dbg.dbg.u.Exception.dwFirstChance:
        return DBG_EXCEPTION_NOT_HANDLED

    crash_bin = utils.crash_binning.crash_binning()
    crash_bin.record_crash(dbg)
    print crash_bin.crash_synopsis()

    dbg.terminate_process()

    return DBG_EXCEPTION_NOT_HANDLED


pid = raw_input("Enter the Process ID: ")
dbg = pydbg()
dbg.attach(int(pid))
dbg.set_callback(EXCEPTION_ACCESS_VIOLATION, check_accessv)
dbg.run()

#### 测试运行：

$\quad\bullet\quad$首先在第一个控制台里运行$\text{buffer_overflow.py}$，此时控制台输出信息：

$\quad\bullet\quad$然后暂时挂起，接着在任务管理器里获取上述$\text{python.exe}$相关联的PID

$\quad\bullet\quad$然后在第二个控制台里运行$\text{access_violation_handler.py}$，并输入上述PID

$\quad\bullet\quad$接着在运行$\text{buffer_overflow.py}$脚本的控制台里输入任意键

$\quad\bullet\quad$然后在运行$\text{access_violation_handler.py}$脚本的控制台里输出了如下几部分信息：

此部分指出了是哪个指令引发的访问异常以及指令在哪个块里；  
这个信息可以帮助写出漏洞利用程序或者用静态分析工具分析问题出在哪里。

此部分转储了所有寄存器的值，特别有趣的是，$\text{EAX}$被覆盖成了$\text{0x41414141}$，而$\text{0x41}$是大写$\text{A}$的十六进制表示。  
同样我们看到$\text{ESI}$指向了一个由$\text{A}$组成的字符串，与$\text{ESP+08}$指向同一个地方。

此部分是在故障指令附近代码的反汇编指令

此部分暂时不知

此部分是奔溃发生时，注册的结构化异常处理程序的列表

用PyDbg构建一个奔溃处理程序就是这么简单，不仅能够自动处理奔溃，还能在时候剖析进程发生的一切。

## 3.进程快照

PyDbg提供了一个非常酷的功能——进程快照。使用进程快照的时候，我们就能冰冻进程，获取进程的内部数据。然后我们想要让进程回到这个时刻的状态时，只需要使用这个时刻的快照就行了。

### 3.1 获得进程快照

第一步需要在某个准确的时间获得一份目标进程的精确快照，为了使得快照足够精确，需要得到所有线程以及CPU的上下文，还有进程的整个内存。将这些信息存储起来，以便需要恢复快照的时候就能用得到。

为了防止在获得快照的时候，进程的数据或者状态被修改，需要将进程挂起来，这个任务由$\text{suspend_all_threads()}$完成。挂起进程后可以用$\text{process_snapshot()}$获取快照。快照完成之后，用$\text{resume_all_threads()}$恢复挂起的进程，让程序继续执行。当某个时刻我们需要将进程恢复到之前的状态，简单的执行$\text{process_restore()}$就可以了。

新建$\text{snapshot.py}$